# 导入GraphLab Create

In [1]:
import graphlab

# 导入CIFAR-10数据集

In [2]:
image_train = graphlab.SFrame('image_train_data/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1493451122.log


This non-commercial license of GraphLab Create for academic use is assigned to heavenyu19820428@yahoo.com and will expire on January 22, 2018.


In [ ]:
deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
deep_learning_model.extract_features(image_train)

In [3]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# 通过应用这些深层特征来构建最近邻模型来进行图像检索

In [4]:
knn_model = graphlab.nearest_neighbors.create(image_train, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

# 通过应用深度特征的图像检索模型找到相似的图像

In [5]:
cat = image_train[18:19]

In [6]:
cat['image'].show()

Canvas is accessible via web browser at the URL: http://localhost:64573/index.html
Opening Canvas in default web browser.


In [7]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 80.767ms     |

| Done         |         | 100         | 379.813ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


In [8]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'], 'id')

In [9]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 43.858ms     |

| Done         |         | 100         | 346.042ms    |

+--------------+---------+-------------+--------------+

In [10]:
cat_neighbors['image'].show()

Canvas is accessible via web browser at the URL: http://localhost:64573/index.html
Opening Canvas in default web browser.


# 找到和轿车相似的图像

In [11]:
car = image_train[8:9]

In [12]:
car['image'].show()

Canvas is accessible via web browser at the URL: http://localhost:64573/index.html
Opening Canvas in default web browser.


In [13]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 43.221ms     |

| Done         |         | 100         | 402.378ms    |

+--------------+---------+-------------+--------------+

Canvas is accessible via web browser at the URL: http://localhost:64573/index.html
Opening Canvas in default web browser.


# 构造一个lambda函数来寻找和显示最近的图像

In [14]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [15]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 49.038ms     |

| Done         |         | 100         | 399.069ms    |

+--------------+---------+-------------+--------------+

Canvas is accessible via web browser at the URL: http://localhost:64573/index.html
Opening Canvas in default web browser.


In [16]:
show_neighbors(18)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 49.856ms     |

| Done         |         | 100         | 345.109ms    |

+--------------+---------+-------------+--------------+

Canvas is accessible via web browser at the URL: http://localhost:64573/index.html
Opening Canvas in default web browser.


In [18]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 48.773ms     |

| Done         |         | 100         | 430.665ms    |

+--------------+---------+-------------+--------------+

Canvas is accessible via web browser at the URL: http://localhost:64573/index.html
Opening Canvas in default web browser.
